In [1]:
# Preamble

from astropy.table import Table, column
import numpy as np
import os

%matplotlib inline
import matplotlib as mpl
from matplotlib import pyplot as plt
# better-looking plots
plt.rcParams['font.family'] = 'serif'
plt.rcParams['figure.figsize'] = (10.0, 8)
plt.rcParams['font.size'] = 18
mpl.ticker.AutoLocator.default_params['nbins'] = 5
mpl.ticker.AutoLocator.default_params['prune'] = 'both'

mpl.rcParams['ps.useafm'] = True
mpl.rcParams['pdf.use14corefonts'] = True
mpl.rcParams['text.usetex'] = True

In [2]:
import sys
sys.path.append('../../../astro_codes/')
#------------------------------------

In [3]:
a70_data = Table.read('../matched_fits/a70_matched.fits')
sfr_data = Table.read('../matched_fits/sfr_table.fits')
chang_out_data = Table.read('../matched_fits/chang_out_matched.fits')
samples = Table.read('../matched_fits/samples.fits')
gas_samples = Table.read('../matched_fits/gas_samples.fits')

In [ ]:
in_vmax = gas_samples['vmax_both']
vmax_weighting = gas_samples['vmax_weighting_both']

in_spiral = (samples['in_spiral']) & (samples['axial_ok']) & (in_vmax)

star_forming = ((samples['star_forming']) & 
                (samples['in_nsa']) & 
                (samples['in_wise']))

In [ ]:
in_sample = (star_forming) & (in_vmax) #& (in_spiral)

gas_mass = a70_data['loghimass']
sfr = sfr_data['total']
mass = chang_out_data['lmass50_all']
ssfr = sfr - mass
weights = gas_samples['vmax_weighting_both']

In [ ]:
from contours import kde_contour

x = gas_mass[in_sample]
y = sfr[in_sample]
w = weights[in_sample]

xy_range = (9,12,-3,2)

_ = kde_contour(x,y,xy_range=xy_range,fill=True,weights=w)
plt.xlabel('$\mathrm{\log(M_{HI})\, (M_\odot)}$')
plt.ylabel('$\mathrm{SFR \, (M_\odot yr^{-1})}$')

_ = plt.axis((9,10.5,-1.5,1))

## Only do this for the M*-limited samples:

In [ ]:
in_mass_limit = samples['in_mass_limit']
in_a70 = gas_samples['in_mass_limit']
in_sample = (in_mass_limit) & (in_a70) & (np.isfinite(sfr_data['total']))
has_spiral = (samples['in_spiral']) & (samples['axial_ok'])
in_spiral = (in_sample) & (has_spiral)

print('M*-limited sample: {} galaxies ({} spirals)'.format(np.sum(in_sample),np.sum(in_spiral)))

sfr = sfr_data['total']
gas_mass = a70_data['loghimass']
mass = chang_out_data['lmass50_all']
weights = gas_samples['vmax_weighting']

_ = plt.hist(gas_mass[in_sample])
#_ = plt.hist(gas_mass[in_sample],weights=weights[in_sample],alpha=0.5)

In [ ]:
from scipy.optimize import curve_fit
from scipy.stats import spearmanr

def linear(x,m,c):
    return m*x + c

x = gas_mass[in_sample]
y = sfr[in_sample]
w = weights[in_sample]

p, _ = curve_fit(linear,x,y)

_ = plt.scatter(x,y,s=w)
_ = kde_contour(x,y)

x_guide = np.linspace(9,11,100)
y_guide = linear(x_guide,*p)
_ = plt.plot(x_guide,y_guide)

_ = plt.axis((9,11,-1.5,1.5))
plt.xlabel('stellar mass')
plt.ylabel('SFR')

print('rho={}'.format(spearmanr(x,y)))

In [ ]:
compare_xy_data = _
from compare_xy import compare_xy_data
help(compare_xy_data)

In [ ]:
colors = ['orange','r','m','g','b']
figlabels = ['1','2','3','4','5+']

x_reference = gas_mass[in_sample]
y_reference = sfr[in_sample]
w_reference = weights[in_sample]
reference_fill_properties = {'alpha':1}

xy_range = (9.9,10.7,-1.5,1.5)
reference_bw = None

for m in range(5):
    
    m_spiral = (in_spiral) & (samples['m'] == m)
    line_properties = {'color':colors[m],
                       'linewidth':2}
    
    x = gas_mass[m_spiral]
    y = sfr[m_spiral]
    w = weights[m_spiral]
    
    fig, axarr, reference_bw = compare_xy_data(x_reference,y_reference,xy_range=xy_range,N_bins=(17,31),
                                               bandwidth=reference_bw,fill=True,contour_fill_properties={'alpha':1},
                                               )#weights=w_reference)
    
    fig, axarr, _ = compare_xy_data(x,y,xy_range=xy_range,N_bins=(17,31),line_properties=line_properties,
                                    fig=fig,axarr=axarr)#,weights=w)

## What fraction of the data is 'detected' in the M*-limited sample?

In [ ]:
from bar_plots import comparison_bar_chart

In [ ]:
a70_targeted = (samples['in_a70_region'])
a70_detected = gas_samples['in_mass_limit']
in_spiral = (samples['in_spiral']) & (samples['axial_ok']) & (samples['in_mass_limit'])

detection_table = Table()

for m in range(5):
    
    m_spiral = (in_spiral) & (samples['m'] == m)
    detected = np.sum(a70_detected[m_spiral])
    print(np.sum(a70_detected[m_spiral]),np.sum(a70_targeted[m_spiral]))
    undetected = np.sum(a70_targeted[m_spiral]) - detected
    detection_table[figlabels[m]] = [detected,undetected]

_ = comparison_bar_chart(detection_table,['in A70','no A70'],colors=['r','w'],normalise='percent',ylabel='$\%$',
                         linewidth=1)

_ = plt.ylim(0,80)
plt.xlabel('m')

In [ ]:
_ = plt.hist(x,bins=np.linspace(9,11,100))
print(x)